# Dataset Plot Accessor Demo (`.fxplot`)

This notebook demonstrates the new `.fxplot` accessor for `xr.Dataset` objects.
It provides convenient Plotly Express plotting methods with smart auto-faceting and coloring.

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr

## Create Sample Data

Let's create a multi-dimensional dataset to demonstrate the plotting capabilities.

In [ ]:
# Simple time-series dataset
np.random.seed(42)
time = pd.date_range('2024-01-01', periods=24, freq='h')

ds_simple = xr.Dataset(
    {
        'Solar': (['time'], np.maximum(0, np.sin(np.linspace(0, 2 * np.pi, 24)) * 50 + np.random.randn(24) * 5)),
        'Wind': (['time'], np.abs(np.random.randn(24) * 20 + 30)),
        'Demand': (['time'], np.abs(np.sin(np.linspace(0, 2 * np.pi, 24) + 1) * 40 + 50 + np.random.randn(24) * 5)),
    },
    coords={'time': time},
)

ds_simple

## Line Plot

In [ ]:
ds_simple.fxplot.line(title='Energy Generation & Demand')

## Stacked Bar Chart

In [ ]:
ds_simple[['Solar', 'Wind']].fxplot.stacked_bar(title='Renewable Generation')

## Area Chart

In [ ]:
ds_simple[['Solar', 'Wind']].fxplot.area(title='Stacked Area - Generation')

## Grouped Bar Chart

In [ ]:
ds_simple.fxplot.bar(title='Grouped Bar Chart')

## Heatmap

In [ ]:
# Create 2D data for heatmap
ds_heatmap = xr.Dataset(
    {
        'temperature': (['day', 'hour'], np.random.randn(7, 24) * 5 + 20),
    },
    coords={
        'day': pd.date_range('2024-01-01', periods=7, freq='D'),
        'hour': range(24),
    },
)

ds_heatmap.fxplot.heatmap('temperature', title='Temperature Heatmap')

## Multi-Dimensional Data with Auto-Faceting

The accessor automatically handles extra dimensions by assigning them to facets or animation based on CONFIG priority.

In [ ]:
# Dataset with scenario dimension
ds_scenarios = xr.Dataset(
    {
        'Solar': (
            ['time', 'scenario'],
            np.column_stack(
                [
                    np.maximum(0, np.sin(np.linspace(0, 2 * np.pi, 24)) * 50),
                    np.maximum(0, np.sin(np.linspace(0, 2 * np.pi, 24)) * 70),  # High scenario
                ]
            ),
        ),
        'Wind': (
            ['time', 'scenario'],
            np.column_stack(
                [
                    np.abs(np.random.randn(24) * 20 + 30),
                    np.abs(np.random.randn(24) * 25 + 40),  # High scenario
                ]
            ),
        ),
    },
    coords={
        'time': time,
        'scenario': ['base', 'high'],
    },
)

ds_scenarios

In [ ]:
# Auto-faceting assigns 'scenario' to facet_col
ds_scenarios.fxplot.line(title='Generation by Scenario (Auto-Faceted)')

In [ ]:
# Explicit faceting
ds_scenarios.fxplot.stacked_bar(facet_col='scenario', title='Stacked Bar by Scenario')

## Animation Support

In [ ]:
# Use animation instead of faceting
ds_scenarios.fxplot.area(facet_col=None, animation_frame='scenario', title='Animated by Scenario')

## Custom Colors

In [ ]:
# Using a colorscale name
ds_simple.fxplot.line(colors='viridis', title='With Viridis Colorscale')

In [ ]:
# Using explicit color mapping
ds_simple.fxplot.stacked_bar(
    colors={'Solar': 'gold', 'Wind': 'skyblue', 'Demand': 'salmon'}, title='With Custom Colors'
)

## Chaining with Plotly Methods

Since all methods return `go.Figure`, you can chain Plotly's update methods.

In [ ]:
(
    ds_simple.fxplot.line(title='Customized Plot')
    .update_layout(xaxis_title='Time of Day', yaxis_title='Power (MW)', legend_title='Source', template='plotly_white')
    .update_traces(line_width=2)
)

## Pre-filtering with xarray

Filter data using xarray methods before plotting.

In [ ]:
# Select specific time range
ds_simple.sel(time=slice('2024-01-01 06:00', '2024-01-01 18:00')).fxplot.line(title='Daytime Only')

In [ ]:
# Select specific variables
ds_simple[['Solar', 'Wind']].fxplot.area(title='Renewables Only')